In [ ]:
!nvidia-smi

Wed Apr 10 06:14:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install weaviate-client langchain tiktoken pypdf rapidocr-onnxruntime

NotImplementedError: A UTF-8 locale is required. Got UtF-8

In [ ]:
WEAVIATE_API_KEY="m6AEHAGJWIOtkE6CBitTA5By0IeKxU08wFxb"
WEAVIATE_CLUSTER="https://mylangchainproject-kgcu2zti.weaviate.network"

In [ ]:
from langchain.vectorstores import Weaviate
import weaviate
WEAVIATE_URL=WEAVIATE_CLUSTER
WEAVIATE_API_KEY=WEAVIATE_API_KEY

client=weaviate.Client(
  url=WEAVIATE_URL,auth_client_secret=weaviate.AuthApiKey(WEAVIATE_API_KEY)
)

In [ ]:
import locale
locale.getpreferredencoding=lambda:"UTF-8"

In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 1.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
embedding_model_name="sentence-transformers/all-mpnet-base-v2"
model_kwargs={"device":"cuda"}
embeddings=HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs=model_kwargs
)

In [ ]:
from langchain.document_loaders import PyPDFLoader
loader=PyPDFLoader("/content/2005.11401.pdf",extract_images=True)
pages=loader.load()

In [ ]:
# pages

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [ ]:
textsplitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=20)

In [ ]:
chunks=textsplitter.split_documents(pages)

In [ ]:
vector_db=Weaviate.from_documents(
    chunks,embeddings,client=client,by_text=False
)

In [ ]:
print(vector_db.similarity_search("What is RAG ?",k=3)[0].page_content)

documents for questions that are less likely to beneﬁt from retrieval, suggesting that null document
mechanisms may not be necessary for RAG.
G Parameters
Our RAG models contain the trainable parameters for the BERT-base query and document encoder of
DPR, with 110M parameters each (although we do not train the document encoder ourselves) and
406M trainable parameters from BART-large, 406M parameters, making a total of 626M trainable
18


In [ ]:
from langchain.prompts import ChatPromptTemplate


In [ ]:
template="""you are an assistant for question-answering tasks .
Use the following pieces of retrieved context to answer the question .
If you don't know the answer,jsut say that you don't know .
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [ ]:
prompt=ChatPromptTemplate.from_template(template)

In [ ]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="you are an assistant for question-answering tasks .\nUse the following pieces of retrieved context to answer the question .\nIf you don't know the answer,jsut say that you don't know .\nUse ten sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:\n"))])

In [ ]:
from langchain import HuggingFaceHub

In [ ]:
from google.colab import userdata
huggingfacehub_api=userdata.get('HUGGINGFACEHUB_API_TOKEN')


In [ ]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN']=api_key

In [ ]:
model=HuggingFaceHub(
    huggingfacehub_api_token=huggingfacehub_api,
    repo_id="mistralai/Mistral-7B-Instruct-v0.1",
    model_kwargs={"temperature":1,"max_length":180}
)

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [ ]:
output_parser=StrOutputParser()

In [ ]:
retriever=vector_db.as_retriever()

In [ ]:
rag_chain=(
    {"context":retriever,"question":RunnablePassthrough()}
    |prompt
    |model
    |output_parser
)

In [ ]:
rag_chain.invoke("Please give the knowledge for vector database ?")

'Human: you are an assistant for question-answering tasks .\nUse the following pieces of retrieved context to answer the question .\nIf you don\'t know the answer,jsut say that you don\'t know .\nUse ten sentences maximum and keep the answer concise.\nQuestion: Please give the knowledge for vector database ?\nContext: [Document(page_content=\'as a target sequence of length one, in which case RAG-Sequence and RAG-Token are equivalent.\\n2.2 Retriever: DPR\\nThe retrieval component pη(z|x)is based on DPR [26]. DPR follows a bi-encoder architecture:\\npη(z|x)∝exp(\\nd(z)⊤q(x))\\nd(z) =BERTd(z),q(x) =BERTq(x)\\nwhere d(z)is a dense representation of a document produced by a BERT BASE document encoder [8],\\nandq(x)a query representation produced by a query encoder , also based on BERT BASE. Calculating\\ntop-k (pη(·|x)), the list ofkdocumentszwith highest prior probability pη(z|x), is a Maximum Inner\\nProduct Search (MIPS) problem, which can be approximately solved in sub-linear time [ 23